In [12]:
import notebook_common
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    # read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    # r_word_2400_xlsx,
    # w_word_2400_csv,
    # w_word_2400_with_guide_csv,
    # w_word_2400_without_guide_csv,
    # ------------------JLPT word------------------
    r_jlpt_csv,
    w_jlpt_csv,
    w_jlpt_word_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    # w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    # w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    # w_merged_freq_sumed_csv,
    w_raw_merged_freq_sumed_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
    re,
    l,
    fetch_random_from_set,
)

l("Import common module")

----------------------------------------Import common module----------------------------------------


In [13]:
# 读取原始 jlpt文件
df_jlpt = pd.read_csv(
    r_jlpt_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    keep_default_na=False,
    # index_col=["word", "reading"]
)

# 把JLPT中的level替换成1到6
df_jlpt['level']=df_jlpt['level'].str[:1]
df_jlpt['level']=df_jlpt['level'].astype('float32')

# 读原始书面语和口语词频合并
df_merged_raw = pd.read_csv(
    w_raw_merged_freq_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    keep_default_na=False,
    # index_col=["headword", "reading"]
)
l("read ok")

----------------------------------------read ok----------------------------------------


In [14]:
l('jlpt')
df_jlpt

----------------------------------------jlpt----------------------------------------


,no,word,reading,level,property,all-property,origin
0,3,亜,ア,6.0,接頭辞,名詞-普通名詞-一般,漢語
1,4,あ,ア,2.0,名詞,名詞-普通名詞-一般,和語
2,5,ああ,アア,2.0,感動詞,感動詞-一般,和語
3,6,ああ,アア,3.0,感動詞,副詞,和語
4,7,アーケード,アーケード,4.0,名詞,名詞-普通名詞-一般,外来語
...,...,...,...,...,...,...,...
17903,20139,部活,ブカツ,5.0,名詞,名詞-普通名詞-一般,漢語
17904,20140,ロケット,ロケット,3.0,名詞,名詞-普通名詞-一般,外来語
17905,20141,離陸,リリク,4.0,名詞,名詞-普通名詞-サ変可能,漢語
17906,20142,踏切,フミキリ,3.0,名詞,名詞-普通名詞-一般,和語


In [15]:
l("df_merged_raw")
df_merged_raw

----------------------------------------df_merged_raw----------------------------------------


,headword,reading,spoken_freq,written_freq
0,の,ノ,57221425.0,686025.0
1,に,ニ,35286930.0,
2,を,ヲ,33707106.0,
3,は,ハ,31279365.0,
4,が,ガ,30705135.0,
...,...,...,...,...
104315,8ミリビデオカメラ,ハチミリビデオカメラ,,1.0
104316,鼻緒する,ハナオスル,,1.0
104317,DJする,ディージェイスル,,1.0
104318,CTB,シーティービー,,1.0


In [16]:
# jlpt中word 找出jlpt词汇里有多少个在词频表里

# 1.jlpt本身的数据
jlpt_shape = df_jlpt.shape
p(f"jlpt shape:{jlpt_shape}")
word_jlpt_set = set(df_jlpt["word"])
p(f"word size in jlpt:{len(word_jlpt_set)}")
reading_jlpt_set = set(df_jlpt["reading"])
p(f"reading size in jlpt:{len(reading_jlpt_set)}")
index_jlpt_set = set(df_jlpt.index)
p(f"index size in jlpt:{len(index_jlpt_set)}")

l()
# 2.合并书面语和口语原始数据
def_merged_raw_shape = df_merged_raw.shape
p(f"def_merged_raw_shape:{def_merged_raw_shape}")

headword_freq_merge_set = set(df_merged_raw["headword"])
p(f"raw merged headword size:{len(headword_freq_merge_set)}")
reading_freq_merge_set = set(df_merged_raw["reading"])
p(f"raw merged reading size:{len(reading_freq_merge_set)}")

jlpt shape:(17908, 7)
word size in jlpt:17207
reading size in jlpt:15592
index size in jlpt:17908
--------------------------------------------------------------------------------
def_merged_raw_shape:(104320, 4)
raw merged headword size:102381
raw merged reading size:80035


In [17]:
# 1. jlpt的word和词频表的headword的交集
word_intersection = word_jlpt_set & headword_freq_merge_set
len_word_jlpt_set = len(word_jlpt_set)
len_word_found = len(word_intersection)
per_word_found = len_word_found / len_word_jlpt_set
len_word_unfound = len_word_jlpt_set - len_word_found
per_word_unfound = len_word_unfound / len_word_jlpt_set

p(f"word size in jlpt:{len_word_jlpt_set}")
p(f"word_intersection:{len_word_found}({per_word_found:.1%})")
p(f"unfound word in jlpt:{len_word_unfound}({per_word_unfound:.1%})")
l()
# 2. jlpt的reading和词频表的reading的交集
reading_intersection = reading_jlpt_set & reading_freq_merge_set
len_reading_jlpt_set = len(reading_jlpt_set)
len_reading_found = len(reading_intersection)
per_reading_found = len_reading_found / len_reading_jlpt_set
len_reading_unfound = len_reading_jlpt_set - len_reading_found
per_reading_unfound = len_reading_unfound / len_reading_jlpt_set
p(f"reading size in jlpt:{len_reading_jlpt_set}")
p(f"reading_intersection:{len_reading_found}({per_reading_found:.1%})")
p(f"unfound reading in jlpt:{len_reading_unfound}({per_reading_unfound:.1%})")
l()
# 3. 计算jlpt中word和reading同时匹配词频
p(f"jlpt shape:{df_jlpt.shape}")
p(f"df_merged_raw shape:{df_merged_raw.shape}")


df_jlpt_with_multiidx = df_jlpt.set_index(["word", "reading"])
df_merged_raw_with_multiidx = df_merged_raw.set_index(["headword", "reading"])

idx_jlbt_set = set(df_jlpt_with_multiidx.index)
idx_df_merged_raw_set = set(df_merged_raw_with_multiidx.index)

l("df_jlpt_with_multiidx.index")
p(f"len df_jlpt_with_multiidx.index:{len(idx_jlbt_set)}")
fetch_random_from_set(idx_jlbt_set)
l("df_merged_raw_with_multiidx.index")
p(f"len idx_df_merged_raw_set.index:{len(idx_df_merged_raw_set)}")
fetch_random_from_set(idx_df_merged_raw_set)
l()
# jlpt的复合索引(word-reading)和词频表的复合索引的交集
index_intersection = idx_jlbt_set & idx_df_merged_raw_set
len_index_jlpt_set = len(idx_jlbt_set)
len_index_found = len(index_intersection)
per_index_found = len_index_found / len_index_jlpt_set
len_index_unfound = len_index_jlpt_set - len_index_found
per_index_unfound = len_index_unfound / len_index_jlpt_set
p(f"index size in jlpt:{len_index_jlpt_set}")
p(f"index_intersection:{len_index_found}({per_index_found:.1%})")
p(f"unfound index in jlpt:{len_index_unfound}({per_index_unfound:.1%})")
l()
df_jlpt_with_freq = df_jlpt.join(
    df_merged_raw, how="left", lsuffix="_jlpt", rsuffix="_freqdict"
)
to_csv_sig(df_jlpt_with_freq, "d:/tmp/h.csv")
p("df_jlpt_with_freq--ok")

word size in jlpt:17207
word_intersection:14753(85.7%)
unfound word in jlpt:2454(14.3%)
--------------------------------------------------------------------------------
reading size in jlpt:15592
reading_intersection:13896(89.1%)
unfound reading in jlpt:1696(10.9%)
--------------------------------------------------------------------------------
jlpt shape:(17908, 7)
df_merged_raw shape:(104320, 4)
----------------------------------------df_jlpt_with_multiidx.index----------------------------------------
len df_jlpt_with_multiidx.index:17496
[('銃', 'ジュウ'), ('猛獣', 'モウジュウ'), ('受かる', 'ウカル'), ('実地', 'ジッチ'), ('インプット', 'インプット')]
----------------------------------------df_merged_raw_with_multiidx.index----------------------------------------
len idx_df_merged_raw_set.index:104320
[('止めれる', 'トメレル'), ('真田', 'サナダ'), ('春闘する', 'シュントウスル'), ('滅種する', 'メツシュスル'), ('天動説', 'テンドウセツ')]
--------------------------------------------------------------------------------
index size in jlpt:17496
index_intersectio

In [18]:
# 取出jlpt中索引不在词频表中的那些数据(复合索引)
df_2873 = df_jlpt_with_multiidx[~df_jlpt_with_multiidx.index.isin(df_merged_raw_with_multiidx.index)]
to_csv_sig(df_2873, "d:/tmp/d.csv", True)
l("df_2873 export to d:/tmp/d.csv")
df_2873

----------------------------------------df_2873 export to d:/tmp/d.csv----------------------------------------


,,no,level,property,all-property,origin
word,reading,,,,,
亜,ア,3,6.0,接頭辞,名詞-普通名詞-一般,漢語
アーチスト,アーチスト,9,3.0,名詞,名詞-普通名詞-一般,外来語
愛煙家,アイエンカ,17,6.0,名詞,名詞-普通名詞-一般 and 接尾辞-名詞的-一般,漢語
ＩＱ,アイキュー,20,5.0,名詞,名詞-普通名詞-一般,外来語
アイスコーヒー,アイスコーヒー,40,1.0,名詞,名詞-普通名詞-一般 and 名詞-普通名詞-一般,外来語
...,...,...,...,...,...,...
お待たせしました,オマタセシマシタ,20110,2.0,定型表現,定型句,定型句
心地よい,ココチヨイ,20130,5.0,イ形容詞,形容詞-一般,和語
焼肉,ヤキニク,20136,2.0,名詞,名詞-普通名詞-一般,混種語


In [19]:
# 取出jlpt中索引不在词频表中的那些数据(只有拼写索引)
df_jlpt_with_word_idx = df_jlpt.set_index("word")
df_merged_raw_with_word_idx = df_merged_raw.set_index("headword")
df_2454 = df_jlpt_with_word_idx[
    ~df_jlpt_with_word_idx.index.isin(df_merged_raw_with_word_idx.index)
]
to_csv_sig(df_2454, "d:/tmp/e.csv", True)
l("df_2454 export to d:/tmp/e.csv")
df_2454

----------------------------------------df_2454 export to d:/tmp/e.csv----------------------------------------


,no,reading,level,property,all-property,origin
word,,,,,,
亜,3,ア,6.0,接頭辞,名詞-普通名詞-一般,漢語
アーチスト,9,アーチスト,3.0,名詞,名詞-普通名詞-一般,外来語
愛煙家,17,アイエンカ,6.0,名詞,名詞-普通名詞-一般 and 接尾辞-名詞的-一般,漢語
ＩＱ,20,アイキュー,5.0,名詞,名詞-普通名詞-一般,外来語
アイスコーヒー,40,アイスコーヒー,1.0,名詞,名詞-普通名詞-一般 and 名詞-普通名詞-一般,外来語
...,...,...,...,...,...,...
お待たせしました,20110,オマタセシマシタ,2.0,定型表現,定型句,定型句
心地よい,20130,ココチヨイ,5.0,イ形容詞,形容詞-一般,和語
焼肉,20136,ヤキニク,2.0,名詞,名詞-普通名詞-一般,混種語


In [20]:
# df_jlpt["word_count"] = df_jlpt.groupby(["word"]).transform("size")
# to_csv_sig(df_jlpt, w_jlpt_csv)
# p("df_jlpt--ok")

In [21]:
to_csv_sig(df_jlpt_with_word_idx,"d:/tmp/f.csv")
p('df_jlpt_with_word_idx--ok')
df_jlpt_with_word_idx

df_jlpt_with_word_idx--ok


,no,reading,level,property,all-property,origin
word,,,,,,
亜,3,ア,6.0,接頭辞,名詞-普通名詞-一般,漢語
あ,4,ア,2.0,名詞,名詞-普通名詞-一般,和語
ああ,5,アア,2.0,感動詞,感動詞-一般,和語
ああ,6,アア,3.0,感動詞,副詞,和語
アーケード,7,アーケード,4.0,名詞,名詞-普通名詞-一般,外来語
...,...,...,...,...,...,...
部活,20139,ブカツ,5.0,名詞,名詞-普通名詞-一般,漢語
ロケット,20140,ロケット,3.0,名詞,名詞-普通名詞-一般,外来語
離陸,20141,リリク,4.0,名詞,名詞-普通名詞-サ変可能,漢語


In [22]:
df_jlpt_with_word_idx.index.name = "word_jlpt"
df_jlpt_word = (
    df_jlpt_with_word_idx.groupby("word_jlpt")
    .agg(
        {
            "reading": lambda x: "|".join(x),
            "property": lambda x: "|".join(x),
            "all-property": lambda x: "|".join(x),
            "origin": lambda x: "|".join(x),
            "level": lambda x: round(x.mean(), 1),
        }
    )  # 字符串拼接  # 求平均值
    .reset_index()
)
df_jlpt_word.rename(
    columns={
        "reading": "reading_jlpt",
        "property": "property_jlpt",
        "all-property": "all_property_jlpt",
        "origin": "origin_jlpt",
    },
    inplace=True,
)
df_jlpt_word.set_index("word_jlpt", inplace=True)
to_csv_sig(df_jlpt_word, w_jlpt_word_csv, True)
p("df_jlpt_word--ok")

df_jlpt_word--ok
